In [ ]:
# MAIN EXECUTION
if __name__ == "__main__":
    print("Starting Blue Bikes Data Transformation Pipeline")
    print("=" * 60)
    
    try:
        # Load your merged data (adjust the path as needed)
        processed_file = PROCESSED_DIR / "rides_weather_combined.parquet"
        
        # Load and validate the data
        merged_data, weather_columns = load_and_validate_merged_data(processed_file)
        
        print(f"\n✓ Successfully loaded and validated data")
        print(f"✓ Using weather columns: {weather_columns}")
        
        # Transform the data with actual weather columns
        print("\n" + "="*60)
        print("TRANSFORMING DATA WITH SLIDING WINDOW")
        print("="*60)
        
        transformed_data = transform_ts_data_into_features_and_target(
            df=merged_data,
            feature_col="ride_count",
            weather_cols=weather_columns,  # Use detected weather columns
            rides_window_size=672,  # 28 days
            weather_window_size=3,   # 3 hours
            step_size=1
        )
        
        # Verify transformation quality
        print("\n" + "="*60)
        print("VERIFYING TRANSFORMATION")
        print("="*60)
        
        # Check for NaN values in transformed data
        null_summary = transformed_data.isnull().sum()
        if null_summary.sum() > 0:
            print("\n⚠️  Warning: Null values found in transformed data:")
            print(null_summary[null_summary > 0])
        else:
            print("✓ No null values in transformed data!")
        
        # Store the transformed data
        print("\n" + "="*60)
        print("STORING TRANSFORMED DATA")
        print("="*60)
        
        final_data = store_transformed_data(transformed_data)
        
        # Analyze the results
        print("\n" + "="*60)
        print("ANALYSIS")
        print("="*60)
        
        analyze_transformed_data(final_data)
        
        print("\n🎉 TRANSFORMATION COMPLETED SUCCESSFULLY!")
        print("The dataset is now ready for machine learning model training.")
        
    except FileNotFoundError as e:
        print(f"❌ File not found: {e}")
        print("\nPlease ensure you have:")
        print("1. Run 03_process_and_save_data.ipynb first")
        print("2. The merged file exists at: data/processed/rides_weather_combined.parquet")
        raise
    except Exception as e:
        print(f"❌ Error in transformation pipeline: {e}")
        import traceback
        traceback.print_exc()
        raise

print("\nCode execution completed!")

Starting Blue Bikes Data Transformation Pipeline
Loading processed data...
Loaded data: 4,950,745 records
Columns: ['hour', 'start_station_id', 'ride_count', 'lat_round', 'lon_round', 'temp', 'humidity', 'pressure', 'wind_speed', 'wind_deg', 'precipitation']

TRANSFORMING DATA WITH SLIDING WINDOW
Starting sliding window transformation...
Rides window size: 672 (28 days)
Weather window size: 3 hours
Step size: 1
Number of stations after filtering: 50
Processing 50 stations...
Processed 10 stations...
Processed 20 stations...
Processed 30 stations...
Processed 40 stations...
Processed 50 stations...
Transformation complete!
Final dataset: 404584 records
Number of features: 681
  - Ride features: 672
  - Weather features: 9

STORING TRANSFORMED DATA
Stored transformed data to: ../data/transformed/transformed_features_and_target_top50.parquet
Dataset shape: (404584, 684)

ANALYSIS

TRANSFORMED DATA ANALYSIS
Total records: 404,584
Number of stations: 50
Time range: 2024-10-28 22:00:00 to 20

In [ ]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Tuple
import warnings
warnings.filterwarnings('ignore')

from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, TRANSFORMED_DATA_DIR

# Set up directories
PROCESSED_DIR = PROCESSED_DATA_DIR
TRANSFORMED_DIR = TRANSFORMED_DATA_DIR

print(f"Processed directory: {PROCESSED_DIR}")
print(f"Transformed directory: {TRANSFORMED_DIR}")

In [ ]:
def load_and_validate_merged_data(processed_file: Path) -> pd.DataFrame:
    """
    Load the merged rides+weather data and perform comprehensive validation
    """
    print("="*60)
    print("LOADING AND VALIDATING MERGED DATA")
    print("="*60)
    
    if not processed_file.exists():
        raise FileNotFoundError(f"Processed file not found: {processed_file}")
    
    print(f"\nLoading: {processed_file.name}")
    df = pd.read_parquet(processed_file)
    
    print(f"Loaded {len(df):,} records")
    print(f"Columns: {list(df.columns)}")
    print(f"Data types:\n{df.dtypes}\n")
    
    # Ensure proper data types
    df['hour'] = pd.to_datetime(df['hour'])
    df['start_station_id'] = df['start_station_id'].astype(str)
    df['ride_count'] = pd.to_numeric(df['ride_count'], errors='coerce').fillna(0).astype(int)
    
    # Identify weather columns
    weather_cols = ["temp", "humidity", "pressure", "wind_speed", "wind_deg", "precipitation"]
    available_weather_cols = [col for col in weather_cols if col in df.columns]
    
    print(f"Available weather columns: {available_weather_cols}")
    
    # Convert weather columns to numeric
    for col in available_weather_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Validate and fill weather data
    df = validate_and_fill_weather(df)
    
    # Final validation
    print("\n" + "-"*60)
    print("FINAL DATA VALIDATION")
    print("-"*60)
    
    print(f"Total records: {len(df):,}")
    print(f"Stations: {df['start_station_id'].nunique()}")
    print(f"Date range: {df['hour'].min()} to {df['hour'].max()}")
    print(f"Ride count - Min: {df['ride_count'].min()}, Max: {df['ride_count'].max()}, Mean: {df['ride_count'].mean():.2f}")
    
    # Check for any remaining null values
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"\n⚠️  Remaining null values:")
        print(null_counts[null_counts > 0])
    else:
        print("\n✓ No null values remaining!")
    
    return df, available_weather_cols

In [ ]:
def transform_ts_data_into_features_and_target(
    df: pd.DataFrame,
    feature_col: str,
    weather_cols: List[str],
    rides_window_size: int = 672,  # 28 days * 24 hours
    weather_window_size: int = 3,  # 3 hours of weather history
    step_size: int = 1
) -> pd.DataFrame:
    """
    Transform time series data into features and target for machine learning.
    Creates lagged features from ride counts and weather data.
    
    Parameters:
    - df: DataFrame with hourly data per station
    - feature_col: Column name for the target variable (e.g., 'ride_count')
    - weather_cols: List of weather column names to include
    - rides_window_size: Number of historical ride hours to include as features
    - weather_window_size: Number of historical weather hours to include as features
    - step_size: Step size for sliding window
    
    Returns:
    - DataFrame with transformed features and target
    """
    print("Starting sliding window transformation...")
    print(f"Rides window size: {rides_window_size} (28 days)")
    print(f"Weather window size: {weather_window_size} hours")
    print(f"Step size: {step_size}")
    
    # Ensure data is sorted by station and hour
    df = df.sort_values(["start_station_id", "hour"]).reset_index(drop=True)
    
    # Get unique stations and filter stations with enough data
    stations = df["start_station_id"].unique()
    min_required_samples = rides_window_size + 100  # Need enough history
    
    # Filter stations with sufficient data
    station_counts = df.groupby("start_station_id").size()
    valid_stations = station_counts[station_counts >= min_required_samples].index.tolist()
    
    # Limit to top 50 stations by ride count for faster processing
    station_totals = df.groupby("start_station_id")[feature_col].sum().sort_values(ascending=False)
    valid_stations = [s for s in station_totals.head(50).index if s in valid_stations]
    
    print(f"Number of stations after filtering: {len(valid_stations)}")
    
    all_transformed_data = []
    
    print(f"Processing {len(valid_stations)} stations...")
    
    for idx, station in enumerate(valid_stations):
        if (idx + 1) % 10 == 0:
            print(f"Processed {idx + 1} stations...")
        
        # Get data for this station
        station_data = df[df["start_station_id"] == station].reset_index(drop=True)
        
        # Extract ride counts and weather features
        ride_values = station_data[feature_col].values
        weather_values = station_data[weather_cols].values if weather_cols else None
        hours = station_data["hour"].values
        
        # Create sliding window features
        for i in range(rides_window_size, len(station_data), step_size):
            # Target is the current hour
            target = ride_values[i]
            
            # Features are historical rides
            ride_features = ride_values[i - rides_window_size:i]
            
            # Add weather features (last N hours)
            if weather_values is not None and weather_window_size > 0:
                weather_features = weather_values[max(0, i - weather_window_size):i].flatten()
            else:
                weather_features = []
            
            # Combine all features
            features = {
                'target': target,
                'start_station_id': station,
                'hour': hours[i]
            }
            
            # Add ride lag features
            for j, val in enumerate(ride_features):
                features[f'rides_t-{rides_window_size - j}'] = val
            
            # Add weather lag features
            if len(weather_features) > 0:
                for j, val in enumerate(weather_features):
                    weather_idx = j // len(weather_cols)
                    col_idx = j % len(weather_cols)
                    features[f'weather_{weather_cols[col_idx]}_t-{weather_window_size - weather_idx - 1}'] = val
            
            all_transformed_data.append(features)
    
    # Create DataFrame
    transformed_df = pd.DataFrame(all_transformed_data)
    
    print("\nTransformation complete!")
    print(f"Final dataset: {len(transformed_df)} records")
    
    # Count features
    ride_features = [col for col in transformed_df.columns if col.startswith('rides_t-')]
    weather_features = [col for col in transformed_df.columns if col.startswith('weather_')]
    
    print(f"Number of features: {len(ride_features) + len(weather_features)}")
    print(f"  - Ride features: {len(ride_features)}")
    print(f"  - Weather features: {len(weather_features)}")
    
    return transformed_df

In [ ]:
def store_transformed_data(transformed_df: pd.DataFrame) -> pd.DataFrame:
    """
    Store the transformed data to the transformed directory
    """
    # Ensure the transformed directory exists
    TRANSFORMED_DIR.mkdir(parents=True, exist_ok=True)
    
    # Save to parquet
    output_file = TRANSFORMED_DIR / "transformed_features_and_target_top50.parquet"
    transformed_df.to_parquet(output_file, index=False)
    
    print(f"Stored transformed data to: {output_file}")
    print(f"Dataset shape: {transformed_df.shape}")
    
    return transformed_df


def analyze_transformed_data(df: pd.DataFrame):
    """
    Analyze the transformed dataset and print summary statistics
    """
    print("\n" + "="*60)
    print("TRANSFORMED DATA ANALYSIS")
    print("="*60)
    
    print(f"Total records: {len(df):,}")
    print(f"Number of stations: {df['start_station_id'].nunique()}")
    print(f"Time range: {df['hour'].min()} to {df['hour'].max()}")
    
    # Count features
    ride_features = [col for col in df.columns if col.startswith('rides_t-')]
    weather_features = [col for col in df.columns if col.startswith('weather_')]
    
    print(f"\nFeature breakdown:")
    print(f"  Ride features: {len(ride_features)}")
    print(f"  Weather features: {len(weather_features)}")
    print(f"  Total features: {len(ride_features) + len(weather_features)}")
    
    # Check for missing values
    missing_count = df.isnull().sum().sum()
    print(f"Missing values: {missing_count}")
    
    # Target statistics
    print(f"\nTarget statistics:")
    print(f"  Min: {df['target'].min()}")
    print(f"  Max: {df['target'].max()}")
    print(f"  Mean: {df['target'].mean():.2f}")
    print(f"  Std: {df['target'].std():.2f}")
    
    # Display sample
    print(f"\nSample of transformed data:")
    display_cols = ['target', 'start_station_id', 'hour'] + ride_features[:5]
    print(df[display_cols].head())

In [ ]:
def validate_and_fill_weather(df: pd.DataFrame) -> pd.DataFrame:
    """
    Validate and fill missing weather data using forward fill and station means
    """
    weather_cols = ["temp", "humidity", "pressure", "wind_speed", "wind_deg", "precipitation"]
    available_weather_cols = [col for col in weather_cols if col in df.columns]
    
    print(f"Validating weather columns: {available_weather_cols}")
    
    for col in available_weather_cols:
        # Count missing before filling
        missing_before = df[col].isnull().sum()
        
        if missing_before > 0:
            # Sort by station and hour for proper forward fill
            df = df.sort_values(["start_station_id", "hour"]).reset_index(drop=True)
            
            # Forward fill by station group
            df[col] = df.groupby("start_station_id")[col].transform(
                lambda x: x.fillna(method='ffill').fillna(method='bfill')
            )
            
            # Fill remaining with station mean
            df[col] = df.groupby("start_station_id")[col].transform(
                lambda x: x.fillna(x.mean())
            )
            
            # Fill any remaining NaN with global mean
            df[col] = df[col].fillna(df[col].mean())
            
            missing_after = df[col].isnull().sum()
            print(f"  {col}: filled {missing_before - missing_after} missing values")
    
    return df